# SuperStore Sales Forecasting 

<p aling='justify'> This analysis provide sales forecasting basing on super store dataset.</p>

Table of contents:
1. Notebook setup and modules import 
2. Data analysis and visualisation 
3. Stationarity analysis 
4. Trend & seasonality analysis
5. Differencing and trend decomposition 
6. AR, MA, ARIMA, SARIMAX
7. Best model selecetion and forecasts
8. Summary

### 1. Notebook setup and modules import

In [74]:
%pip install Config/requirements.txt

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'Config/requirements.txt'


In [75]:
%load_ext sql 

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [76]:
%sql mssql+pyodbc://PLLUKOR1@local

In [77]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import matplotlib.ticker as ticker

In [78]:
%sql use SuperStore;

 * mssql+pyodbc://PLLUKOR1@local
Done.


[]

### 2. Data Analysis and Visualisation 

<p aling='justify'>To perform data preprocessing and time series forecasting properly, sufficient amount of sample is required. During the differencing process some cases are lost, so it is important to have as many data samples as possible with high flexibility according to generealization. SuperSotre dataset includes sales data for range of 4 years. Chosing proper basis is a key factor for time series forecasting and it is first step to do before kicking-off forecasting task. <p> 

In [ ]:
query = '''
'''